In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix
)

import joblib

In [2]:
df = pd.read_csv("Loan_status_2007-2020Q3.gzip", low_memory=False)
df.head()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2,1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,3,1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,4,1075358,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [3]:
df.shape

(2925493, 142)

In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 2925493 entries, 0 to 2925492
Columns: 142 entries, Unnamed: 0 to debt_settlement_flag
dtypes: float64(106), int64(1), str(35)
memory usage: 3.1 GB


In [5]:
df['loan_status'].value_counts()

loan_status
Fully Paid                                             1497783
Current                                                1031016
Charged Off                                             362548
Late (31-120 days)                                       16154
In Grace Period                                          10028
Late (16-30 days)                                         2719
Issued                                                    2062
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                    433
Name: count, dtype: int64

In [6]:
print("Shape:", df.shape)
print("\nLoan Status Distribution:\n")
print(df['loan_status'].value_counts())


Shape: (2925493, 142)

Loan Status Distribution:

loan_status
Fully Paid                                             1497783
Current                                                1031016
Charged Off                                             362548
Late (31-120 days)                                       16154
In Grace Period                                          10028
Late (16-30 days)                                         2719
Issued                                                    2062
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                    433
Name: count, dtype: int64


In [7]:
# Keep only final outcomes
df_filtered = df[df['loan_status'].isin([
    'Fully Paid',
    'Charged Off',
    'Default'
])]

print("New Shape:", df_filtered.shape)
print("\nNew Loan Status Distribution:\n")
print(df_filtered['loan_status'].value_counts())

New Shape: (1860764, 142)

New Loan Status Distribution:

loan_status
Fully Paid     1497783
Charged Off     362548
Default            433
Name: count, dtype: int64


In [8]:
df_filtered['target'] = df_filtered['loan_status'].apply(
    lambda x: 0 if x == 'Fully Paid' else 1
)

print(df_filtered['target'].value_counts(normalize=True))

target
0    0.804929
1    0.195071
Name: proportion, dtype: float64


/tmp/ipykernel_14303/3606808747.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_filtered['target'] = df_filtered['loan_status'].apply(


In [9]:
print(df_filtered.columns.tolist())


['Unnamed: 0', 'id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_c

In [10]:
leakage_cols = [
    'loan_status',  # we already created target
    'out_prncp',
    'out_prncp_inv',
    'total_pymnt',
    'total_pymnt_inv',
    'total_rec_prncp',
    'total_rec_int',
    'total_rec_late_fee',
    'recoveries',
    'collection_recovery_fee',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'last_fico_range_high',
    'last_fico_range_low',
    'hardship_flag',
    'hardship_type',
    'hardship_reason',
    'hardship_status',
    'hardship_amount',
    'hardship_start_date',
    'hardship_end_date',
    'payment_plan_start_date',
    'hardship_length',
    'hardship_dpd',
    'hardship_loan_status',
    'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount',
    'hardship_last_payment_amount',
    'debt_settlement_flag'
]

df_model = df_filtered.drop(columns=leakage_cols)

print("New shape after leakage removal:", df_model.shape)

New shape after leakage removal: (1860764, 112)


In [11]:
drop_cols = [
    'Unnamed: 0',
    'id',
    'url',
    'emp_title',
    'title',
    'zip_code'
]

df_model = df_model.drop(columns=drop_cols)

print("Shape after removing ID/text columns:", df_model.shape)


Shape after removing ID/text columns: (1860764, 106)


In [12]:
df_model.info()

<class 'pandas.DataFrame'>
Index: 1860764 entries, 0 to 2925492
Columns: 106 entries, loan_amnt to target
dtypes: float64(88), int64(1), str(17)
memory usage: 1.5 GB


In [13]:
# Check missing value percentage
missing_percent = df_model.isnull().mean().sort_values(ascending=False) * 100
print(missing_percent.head(20))

deferral_term                         99.317108
sec_app_revol_util                    96.899768
revol_bal_joint                       96.847048
sec_app_num_rev_accts                 96.846994
sec_app_mort_acc                      96.846994
sec_app_inq_last_6mths                96.846994
sec_app_collections_12_mths_ex_med    96.846994
sec_app_earliest_cr_line              96.846994
sec_app_chargeoff_within_12_mths      96.846994
sec_app_fico_range_high               96.846994
sec_app_open_acc                      96.846994
sec_app_open_act_il                   96.846994
sec_app_fico_range_low                96.846994
verification_status_joint             96.381110
dti_joint                             96.263255
annual_inc_joint                      96.263094
mths_since_last_record                83.414071
mths_since_recent_bc_dlq              76.572580
mths_since_last_major_derog           73.879009
mths_since_recent_revol_delinq        66.797509
dtype: float64


In [14]:
# Drop columns with more than 90% missing values
high_missing_cols = missing_percent[missing_percent > 90].index.tolist()

df_model = df_model.drop(columns=high_missing_cols)

print("Shape after dropping >90% missing columns:", df_model.shape)

Shape after dropping >90% missing columns: (1860764, 90)


In [15]:
df_model.isnull().mean().sort_values(ascending=False).head(15) * 100


mths_since_last_record            83.414071
mths_since_recent_bc_dlq          76.572580
mths_since_last_major_derog       73.879009
mths_since_recent_revol_delinq    66.797509
il_util                           53.127103
mths_since_last_delinq            50.802520
mths_since_rcnt_il                47.048363
all_util                          45.496473
inq_last_12m                      45.489057
total_cu_tl                       45.489057
open_acc_6m                       45.489057
open_act_il                       45.489003
open_il_24m                       45.489003
max_bal_bc                        45.489003
open_rv_24m                       45.489003
dtype: float64

In [16]:
num_cols = df_model.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()

print("Number of numerical columns:", len(num_cols))
print("Number of categorical columns:", len(cat_cols))
print("\nCategorical Columns:")
print(cat_cols)

Number of numerical columns: 75
Number of categorical columns: 15

Categorical Columns:
['term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'purpose', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'application_type']


/tmp/ipykernel_14303/3758103590.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()


In [17]:
print(df_model['term'].dtype)
print(df_model['int_rate'].dtype)
print(df_model['revol_util'].dtype)

str
str
str


In [18]:
if df_model['term'].dtype == 'object':
    df_model['term'] = df_model['term'].str.extract(r'(\d+)').astype(float)

if df_model['int_rate'].dtype == 'object':
    df_model['int_rate'] = df_model['int_rate'].str.replace('%', '', regex=False).astype(float)

if df_model['revol_util'].dtype == 'object':
    df_model['revol_util'] = df_model['revol_util'].str.replace('%', '', regex=False).astype(float)

In [19]:
num_cols = df_model.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()

print("Numerical:", len(num_cols))
print("Categorical:", len(cat_cols))
print(cat_cols)


Numerical: 75
Categorical: 15
['term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'purpose', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'application_type']


/tmp/ipykernel_14303/2818119915.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()


In [20]:
# Convert to datetime
df_model['issue_d'] = pd.to_datetime(df_model['issue_d'], format='%b-%Y')
df_model['earliest_cr_line'] = pd.to_datetime(df_model['earliest_cr_line'], format='%b-%Y')

# Create credit history length in years
df_model['credit_history_years'] = (
    (df_model['issue_d'] - df_model['earliest_cr_line']).dt.days / 365
)

# Drop original date columns
df_model = df_model.drop(columns=['issue_d', 'earliest_cr_line'])

print("Shape after date processing:", df_model.shape)

Shape after date processing: (1860764, 89)


In [21]:
cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()
print("Categorical:", len(cat_cols))
print(cat_cols)


Categorical: 13
['term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 'addr_state', 'revol_util', 'initial_list_status', 'application_type']


/tmp/ipykernel_14303/1154214966.py:1: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = df_model.select_dtypes(include=['object']).columns.tolist()


In [22]:
df_model = df_model.drop(columns=['grade'])
print("Shape after dropping grade:", df_model.shape)

Shape after dropping grade: (1860764, 88)


In [23]:
print(df_model['pymnt_plan'].value_counts())

pymnt_plan
n    1860764
Name: count, dtype: int64


In [24]:
df_model = df_model.drop(columns=['pymnt_plan'])
print("Shape after dropping pymnt_plan:", df_model.shape)

Shape after dropping pymnt_plan: (1860764, 87)


In [25]:
desired_columns = ['annual_inc', 'loan_amnt', 'credit_history_years', 'home_ownership']

X = df_model[desired_columns]
y = df_model['target']

print("Features selected:", desired_columns)
print("New Feature Shape:", X.shape)

Features selected: ['annual_inc', 'loan_amnt', 'credit_history_years', 'home_ownership']
New Feature Shape: (1860764, 4)


In [26]:
df_model.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'sub_grade', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'collections_12_mths_ex_med',
       'mths_since_last_major_derog', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_a

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train distribution:\n", y_train.value_counts(normalize=True))
print("y_test distribution:\n", y_test.value_counts(normalize=True))

X_train: (1488611, 4)
X_test: (372153, 4)
y_train distribution:
 target
0    0.804929
1    0.195071
Name: proportion, dtype: float64
y_test distribution:
 target
0    0.80493
1    0.19507
Name: proportion, dtype: float64


In [28]:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print("Numerical columns:", len(num_cols))
print("Categorical columns:", len(cat_cols))

Numerical columns: 3
Categorical columns: 1


/tmp/ipykernel_14303/1644831316.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()


In [29]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Numeric preprocessing
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Categorical preprocessing
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, num_cols),
        ('cat', categorical_pipeline, cat_cols)
    ]
)

# Full pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        n_jobs=-1
    ))
])

In [30]:
model_pipeline.fit(X_train, y_train)

/home/prashant/Code/Data Science/CodeAlpha/Credit Risk Model/venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('classifier', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transforme

In [31]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Predict probabilities
y_pred_proba = model_pipeline.predict_proba(X_test)[:, 1]

# Predict class labels
y_pred = model_pipeline.predict(X_test)

# ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("ROC-AUC Score:", roc_auc)

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

ROC-AUC Score: 0.6012905948383813

Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.55      0.67    299557
           1       0.24      0.59      0.34     72596

    accuracy                           0.56    372153
   macro avg       0.54      0.57      0.51    372153
weighted avg       0.73      0.56      0.61    372153


Confusion Matrix:

[[166082 133475]
 [ 29989  42607]]


In [36]:
import joblib

joblib.dump(model_pipeline, "credit_risk_model.pkl")

['credit_risk_model.pkl']